In [1]:
%matplotlib inline
import pandas as pd
import json
import os
from statsbombpy import sb
from mplsoccer import Pitch, Sbopen
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np

In [2]:
euros_2024_competition_id = 55
euros_2024_season_id = 282
matches = sb.matches(competition_id=euros_2024_competition_id, season_id=euros_2024_season_id)
spain_matches = matches[(matches['home_team'] == 'Spain') | (matches['away_team'] == 'Spain')]

In [10]:
olmo_shot_data = []

for _, match in spain_matches.iterrows():
    match_id = match['match_id']
    home_team = match['home_team']
    away_team = match['away_team']
    
    parser = Sbopen()
    df, related, freeze, tactics = parser.event(match_id)
    
    df_olmo_shots = df[(df['player_name'] == 'Daniel Olmo Carvajal') & (df['type_name'] == 'Shot')]
    
    if not df_olmo_shots.empty:
        for _, shot in df_olmo_shots.iterrows():
            xG = shot['shot_statsbomb_xg']
            outcome = shot['outcome_name']
            
            olmo_shot_data.append({
                'Player': 'Daniel Olmo Carvajal',
                'Home Team': home_team,
                'Away Team': away_team,
                'xG': xG,
                'Outcome': outcome
            })

df_olmo_shots = pd.DataFrame(olmo_shot_data, columns=['Player', 'Home Team', 'Away Team', 'xG', 'Outcome'])

average_xG = df_olmo_shots['xG'].mean()

df_olmo_shots['Avg xG'] = average_xG

print(df_olmo_shots)

print(f"Average xG for Dani Olmo: {average_xG:.2f}")
outcome_counts = df_olmo_shots['Outcome'].value_counts()

outcome_summary = pd.DataFrame({
    'Outcome': outcome_counts.index,
    'Count': outcome_counts.values
})

print("\nOutcome Summary:")
print(outcome_summary)


                  Player Home Team Away Team        xG  Outcome    Avg xG
0   Daniel Olmo Carvajal     Spain   England  0.026549  Blocked  0.081065
1   Daniel Olmo Carvajal     Spain   England  0.245579    Off T  0.081065
2   Daniel Olmo Carvajal     Spain    France  0.073872     Goal  0.081065
3   Daniel Olmo Carvajal     Spain   Germany  0.029002    Saved  0.081065
4   Daniel Olmo Carvajal     Spain   Germany  0.163165     Goal  0.081065
5   Daniel Olmo Carvajal     Spain   Germany  0.018918  Blocked  0.081065
6   Daniel Olmo Carvajal     Spain   Georgia  0.030946  Blocked  0.081065
7   Daniel Olmo Carvajal     Spain   Georgia  0.071500  Wayward  0.081065
8   Daniel Olmo Carvajal     Spain   Georgia  0.266806    Saved  0.081065
9   Daniel Olmo Carvajal     Spain   Georgia  0.128134     Goal  0.081065
10  Daniel Olmo Carvajal     Spain   Georgia  0.053031    Saved  0.081065
11  Daniel Olmo Carvajal     Spain   Georgia  0.061233  Blocked  0.081065
12  Daniel Olmo Carvajal   Albania    

In [12]:
england_matches = matches[(matches['home_team'] == 'England') | (matches['away_team'] == 'England')]
kane_shot_data = []
for _, match in england_matches.iterrows():
    match_id = match['match_id']
    home_team = match['home_team']
    away_team = match['away_team']
    
    parser = Sbopen()
    df, related, freeze, tactics = parser.event(match_id)
    
    df_kane_shots = df[(df['player_name'] == 'Harry Kane') & (df['type_name'] == 'Shot')]
    
    if not df_kane_shots.empty:
        for _, shot in df_kane_shots.iterrows():
            xG = shot['shot_statsbomb_xg']
            outcome = shot['outcome_name']
            
            kane_shot_data.append({
                'Player': 'Harry Kane',
                'Home Team': home_team,
                'Away Team': away_team,
                'xG': xG,
                'Outcome': outcome
            })

df_kane_shots = pd.DataFrame(kane_shot_data, columns=['Player', 'Home Team', 'Away Team', 'xG', 'Outcome'])

average_xG = df_kane_shots['xG'].mean()

df_kane_shots['Avg xG'] = average_xG

print(df_kane_shots)

print(f"Average xG for Kane: {average_xG:.2f}")
outcome_counts = df_kane_shots['Outcome'].value_counts()

outcome_summary = pd.DataFrame({
    'Outcome': outcome_counts.index,
    'Count': outcome_counts.values
})

print("\nOutcome Summary:")
print(outcome_summary)


        Player    Home Team    Away Team        xG        Outcome    Avg xG
0   Harry Kane  Netherlands      England  0.028932          Saved  0.169658
1   Harry Kane  Netherlands      England  0.783500           Goal  0.169658
2   Harry Kane        Spain      England  0.048372        Blocked  0.169658
3   Harry Kane      Denmark      England  0.052687        Blocked  0.169658
4   Harry Kane      Denmark      England  0.454891           Goal  0.169658
5   Harry Kane      England  Switzerland  0.024831        Wayward  0.169658
6   Harry Kane      England  Switzerland  0.043730        Blocked  0.169658
7   Harry Kane      England     Slovakia  0.051737        Blocked  0.169658
8   Harry Kane      England     Slovakia  0.108595        Blocked  0.169658
9   Harry Kane      England     Slovakia  0.161387          Off T  0.169658
10  Harry Kane      England     Slovakia  0.121842          Off T  0.169658
11  Harry Kane      England     Slovakia  0.497244           Goal  0.169658
12  Harry Ka